In [ ]:
# !pip install torch==1.7.1
# !pip install transformers

In [ ]:
# !pip install tensorflow
# !pip install setuptools==69.5.1
# !pip install --upgrade pip setuptools wheel
# !pip install numpy --only-binary :all

In [42]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
# import gluonnlp as nlp
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [9]:
import tensorflow as tf
tf.__version__

'2.10.0'

In [10]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
print(device)

cuda


In [11]:
# file_path = '/content/신장내과_본원_외래_20230701_20231231_filtered.xlsx'
file_path = "D:\\sample\\content\\신장내과_본원_외래_20230701_20231231_filtered.xlsx"

# selected_columns = ['ICD10', 'transstr']
selected_columns = ['ICD10', '주호소', 'PI_trans']
df = pd.read_excel(file_path, usecols=selected_columns, dtype=str)

print('전체 데이터 수 :', len(df))
df.head(5)

전체 데이터 수 : 826


,ICD10,주호소,PI_trans
0,N391,NaN,115-61-61 2023.5.10.Suspicion of proteinuria i...
1,N184,"CKD4, Cr 2.26, eGFR 29ml/min","130/61/69 Symptoms (-), edema (-), urbanic sym..."
2,N184,CKD4,"153/75/85 osophy (-), edema (-), urinary sympt..."
3,N185,"Cr 3.74, eGFR 13.56",148/90/64 Idema Mild CKD D/T ADPKD has been tr...
4,N183,lab abnormality,"Hello teacher, I am a patient who is being hos..."


In [12]:
pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [13]:
# 번역
import googletrans

def trans_hng_to_eng(text):
    translator = googletrans.Translator()
    rtn = ''
    # if math.isnan(float(text)):
    if not pd.isna(text):
        text = text.strip()
        text = translator.translate(text, dest='en')
        rtn = text.text
    return rtn

In [95]:
# trans_hng_to_eng(df['주호소'][1])

'CKD4, Cr 2.26, eGFR 29ml/min'

In [14]:
# df['transstr'] = df['현병력'].apply(trans_hng_to_eng)
# df['transstr'][:5]

df['CC_trans'] = df['주호소'].apply(trans_hng_to_eng)
df['CC_trans'][:5]

0                                
1    CKD4, Cr 2.26, eGFR 29ml/min
2                            CKD4
3             Cr 3.74, eGFR 13.56
4                 lab abnormality
Name: CC_trans, dtype: object

In [20]:
def combine_2rd_columns(col_1, col_2):
    result = col_1.strip()
    if not pd.isna(col_2):
        result += " " + str(col_2).strip()
    return result

df["record"] = df.apply(lambda x: combine_2rd_columns(x['CC_trans'], x['PI_trans']), axis=1)

In [26]:
# df.apply
# df['record'].head(10)
df

,ICD10,주호소,PI_trans,CC_trans,record
0,N391,NaN,115-61-61 2023.5.10.Suspicion of proteinuria i...,,115-61-61 2023.5.10.Suspicion of proteinuria ...
1,N184,"CKD4, Cr 2.26, eGFR 29ml/min","130/61/69 Symptoms (-), edema (-), urbanic sym...","CKD4, Cr 2.26, eGFR 29ml/min","CKD4, Cr 2.26, eGFR 29ml/min 130/61/69 Symptom..."
2,N184,CKD4,"153/75/85 osophy (-), edema (-), urinary sympt...",CKD4,"CKD4 153/75/85 osophy (-), edema (-), urinary ..."
3,N185,"Cr 3.74, eGFR 13.56",148/90/64 Idema Mild CKD D/T ADPKD has been tr...,"Cr 3.74, eGFR 13.56","Cr 3.74, eGFR 13.56 148/90/64 Idema Mild CKD D..."
4,N183,lab abnormality,"Hello teacher, I am a patient who is being hos...",lab abnormality,"lab abnormality Hello teacher, I am a patient ..."
...,...,...,...,...,...
821,N183,Azotemia,136/73-73 23.02 EDEMA occurs after shoulder su...,Azotemia,Azotemia 136/73-73 23.02 EDEMA occurs after sh...
822,N184,Azotemia,The patient is a diagnostic [DJD] patient of 8...,Azotemia,Azotemia The patient is a diagnostic [DJD] pat...
823,N185,konos 등록,"KONOS Registration HD 3 times a week, gold at ...",KONOS registration,KONOS registration KONOS Registration HD 3 tim...
824,R601,Generalized edema,He has been edema two months ago.Urine comes o...,Generalized edema,Generalized edema He has been edema two months...


In [24]:
max_len = 0

for rowstr in df['record']:
    if len(rowstr) > max_len:
        max_len = len(rowstr)

print(max_len)

1247


In [25]:
# 학습용, 테스트용, 검증용 데이터 분할
X_train, X_temp, Y_train, Y_temp = train_test_split(df['record'], df['ICD10'], test_size=0.3, shuffle=True, random_state=52)
X_test, X_val, Y_test, Y_val = train_test_split(X_temp, Y_temp, test_size=0.5, shuffle=True, random_state=73)

print(X_train)
print("=======================================\n")
print(X_test)
print("=======================================\n")
print(X_val)
print("=======================================\n")
print(Y_train)
print("=======================================\n")
print(Y_test)
print("=======================================\n")
print(Y_val)

691    persistent proteinuria and microscopic hematur...
190    consultation for operation # A 3.3cm Cystic le...
217    Cr 1.43 129/76/85 Patients with LMC with an ED...
492    Proteinuria 7-8 years ago I heard that there i...
674    Azotemia 98/67-91 New Dongsan Hospital dementi...
                             ...                        
86     for evaluation of renal function HTN 12 years,...
151    Reminery decrease, hyperkalemia # HTN (2022.)2...
525    for kidney donation Visit to donation for kidn...
779    Proteinuria and hematuria since 2023.2 135/94/...
668     The patient was hospitalized with CVA and was...
Name: record, Length: 578, dtype: object

4      lab abnormality Hello teacher, I am a patient ...
432    azotemia The patient is the 84 -year -old (m) ...
272    hematuria 2023. For the first time in the exam...
342    Frequent urination The patient is a diagnostic...
56      139-77-58 Gunjin Award 2023.6.28.Creatinine 2...
                             ...              

In [ ]:
#https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

In [28]:
# Tokenizer
# tokenizer = AutoTokenizer.from_pretrained('NitzanBar/umls-spanbert')
tokenizer = AutoTokenizer.from_pretrained('GanjinZero/UMLSBert_ENG')

# Model
# model = AutoModelForSequenceClassification.from_pretrained('NitzanBar/umls-spanbert')
model = AutoModelForSequenceClassification.from_pretrained('GanjinZero/UMLSBert_ENG', num_labels=14)
model.to(device)

# Vocabulary
vocab = tokenizer.get_vocab()

# Tokenize
tok = tokenizer.tokenize

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GanjinZero/UMLSBert_ENG and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [108]:
class EMRDiagcdDataset(Dataset):

    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        record = self.df.iloc[index]['record']
        # diagcd = self.df.iloc[index]['ICD10']
        diagcd = self.df.iloc[index]['encoded_ICD10']

        return record, diagcd

In [29]:
class EMRDiagcdDataset2(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [31]:
# label encoding
unique_label = df['ICD10'].unique()
print("labels :", unique_label)

label_encoder = LabelEncoder()
label_encoder.fit(unique_label)

# Training dataset labels
temp_df = Y_train.to_frame()
temp_df['encoded_ICD10'] = label_encoder.transform(temp_df['ICD10'])
Y_train_encoding = pd.Series(temp_df['encoded_ICD10'])

# Test dataset labels
temp_df = Y_test.to_frame()
temp_df['encoded_ICD10'] = label_encoder.transform(temp_df['ICD10'])
Y_test_encoding = pd.Series(temp_df['encoded_ICD10'])

labels : ['N391' 'N184' 'N185' 'N183' 'N029' 'N182' 'N178' 'R798' 'R808' 'R311'
 'N059' 'Z524' 'N028' 'R601']


In [32]:
max_len = 1500
batch_size = 32
warmup_ratio = 0.1
num_epochs = 1  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [34]:
# Training data loader
train_dataset = EMRDiagcdDataset2(X_train, 0, 1, tok, vocab, max_len, True, False)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_worker=5)

# Test data loader
test_dataset = EMRDiagcdDataset2(X_test, 0, 1, tok, vocab, max_len, True, False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=false, num_worker=5)

NameError: name 'nlp' is not defined

In [110]:
# Training data loader
train_df = pd.concat([X_train, Y_train_encoding, Y_train], axis=1)
train_dataset = EMRDiagcdDataset2(train_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Test data loader
test_df = pd.concat([X_test, Y_test_encoding, Y_test], axis=1)
test_dataset = EMRDiagcdDataset2(test_df)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=false)

In [39]:
# train_df.to_excel('D:\\sample\\content\\train_df.xlsx')
df.to_excel('D:\\sample\\content\\translated_df.xlsx')

In [ ]:
# class UmlsBERTClassifier(nn.Module):
#     def __init__(self,
#                  bert,
#                  hidden_size = 768,
#                  num_classes=14,   ##클래스 수 조정##
#                  dr_rate=None,
#                  params=None):
#         super(UmlsBERTClassifier, self).__init__()
#         self.bert = bert
#         self.dr_rate = dr_rate
                 
#         self.classifier = nn.Linear(hidden_size , num_classes)
#         if dr_rate:
#             self.dropout = nn.Dropout(p=dr_rate)
    
#     def gen_attention_mask(self, token_ids, valid_length):
#         attention_mask = torch.zeros_like(token_ids)
#         for i, v in enumerate(valid_length):
#             attention_mask[i][:v] = 1
#         return attention_mask.float()

#     def forward(self, token_ids, valid_length, segment_ids):
#         attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
#         _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
#         if self.dr_rate:
#             out = self.dropout(pooler)
#         return self.classifier(out)

In [ ]:
# #BERT 모델 불러오기
# model = UmlsBERTClassifier(model,  dr_rate=0.5).to(device)
 
# #optimizer와 schedule 설정
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

# t_total = len(train_dataloader) * num_epochs
# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# #정확도 측정을 위한 함수 정의
# def calc_accuracy(X,Y):
#     max_vals, max_indices = torch.max(X, 1)
#     train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
#     return train_acc
    
# train_dataloader

In [121]:
optimizer = Adam(model.parameters(), lr=1e-2)
itr = 1
p_itr = 2
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

In [128]:
# train
model.train()

for epoch in range(epochs):
    for record, diagcd in train_loader:
        optimizer.zero_grad()
        # print("record: ", record)
        # print("diagcd: ", diagcd)
        encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
        encoded_record = encoded_record.to(device)
        # print(type(encoded_record))
        diagcd = torch.tensor(diagcd).long().to(device)
        # diagcd = torch.tensor(list(diagcd), dim=0).to(device)

        outputs = model(**encoded_record, labels=diagcd)
        
        loss = outputs.loss
        logits = outputs.logits

        # print('lsos : ', loss)
        # print('record : ', record)
        print('encoded_record : ', encoded_record)
        print('diagcd : ', diagcd)
        print('logits : ', logits)
        
        predict = torch.argmax(F.softmax(logits), dim=0)
        print('predict : ', predict)
        print('predict sum: ', predict.sum())
        correct = predict.eq(diagcd)
        print('correct : ', correct)
        total_correct += correct.sum().item()
        total_len += len(diagcd)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr += 1

C:\Users\KLab\AppData\Local\Temp\ipykernel_29896\3858523646.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  diagcd = torch.tensor(diagcd).long().to(device)


encoded_record :  {'input_ids': tensor([[    2, 18846,  1828,  ...,     0,     0,     0],
        [    2,  3326,    17,  ...,     0,     0,     0],
        [    2, 11016, 10290,  ...,     0,     0,     0],
        ...,
        [    2,  2784, 10281,  ...,     0,     0,     0],
        [    2,  1725,  3517,  ...,     0,     0,     0],
        [    2, 21047,    15,  ...,     0,     0,     0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
diagcd :  tensor([ 5,  9,  7,  4,  6,  5,  7,  5,  9,  5,  3,  8,  1, 12,  5,  5

C:\Users\KLab\AppData\Local\Temp\ipykernel_29896\3858523646.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predict = torch.argmax(F.softmax(logits), dim=0)


RuntimeError: The size of tensor a (14) must match the size of tensor b (32) at non-singleton dimension 0

                                              transstr  encoded_ICD10 ICD10
278  114-69-88 2 years ago Urology and Browno Holy ...              9  R311
690  97-53-112 Diabetes and Hypertension-more than ...              5  N183
108  145-75-85 2023.4.Microscopic Hematuria 2022.10...              9  R311
499  2023/04 I heard that there is proteinuria duri...              9  R311
755  The patient is a patient of 77 years old, curr...              5  N183
..                                                 ...            ...   ...
228  The patient is a diagnostic name [Paralysis of...              5  N183
151  # HTN (2022.)2023. CR 1.24 EGFR 41 EGFR K 5.7 ...              5  N183
129  The patient is a diagnostic [DJD] patient of 8...              5  N183
781  We will ask for consultation.The patient is a ...              9  R311
159  20 years ago, DM, HTN diagnosis 10 years ago, ...              5  N183

[83 rows x 3 columns]


In [33]:
# evaluation
model.eval()

total_len = 0
total_correct = 0

for record, diagcd in test_loader:

    encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
    encoded_record, diagcd = encoded_record.to(device), diagcd.long().to(device)

    outputs = model(**encoded_record, labels=diagcd)

    logits = outputs.logits

    predict = torch.argmax(F.softmax(logits), dim=1)
    correct = predict.eq(diagcd)

    total_correct += correct.sum().item()
    total_len += len(diagcd)

print('Test accuracy: ', total_correct / total_len)

C:\Users\KLab\AppData\Local\Temp\ipykernel_88648\1370849160.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predict = torch.argmax(F.softmax(logits), dim=1)


Test accuracy:  0.0963855421686747
